In [70]:
!pip install us pandas

In [71]:
import os 
import pandas as pd
import calendar
import us

In [72]:
data_file = os.path.join('../data/banned_books.csv')
data = pd.read_csv(data_file)

In [73]:
# convert ID field to numbers (do I really need it)
data["ID"] = pd.to_numeric(data["ID"])

In [74]:
# Drop columns without a lot of data
data = data.drop("Links1", axis=1)
data = data.drop("Links2", axis=1)
data = data.drop("Links3", axis=1)
data = data.drop("Links4", axis=1)
data = data.drop("Police", axis=1)
data = data.drop("Illustrators", axis=1)
data = data.drop("Book_Co_Author", axis=1)
data = data.drop("Restrictions", axis=1)
data = data.drop("Restriction_Details", axis=1)
data = data.drop("Overseeing_Agency", axis=1)
data = data.drop("Last_Updated", axis=1)
data = data.drop("Initial_Response", axis=1)
data = data.drop("Challenge_Notes", axis=1)
data = data.drop("Sch_Name", axis=1)
data = data.drop("Lib_name", axis=1)
data = data.drop("Data_Entry_Date", axis=1)
data = data.drop("Series_Name", axis=1)
data = data.drop("City", axis=1)
data = data.drop("County", axis=1)
data = data.drop("Book_Status", axis=1)
# data = data.drop("ID", axis=1)
# data = data.drop("Challenge_Type", axis=1)
# data = data.drop("Challenge_Init", axis=1)



display(data.columns)

Index(['ID', 'New_Book_Author', 'New_Book_Title', 'Challenge_Type',
       'Challenge_Init', 'Decision', 'Library_Type', 'Month', 'Year', 'State'],
      dtype='object')

In [75]:
# change unset values to Unclear in challenge type
data["Challenge_Type"] = data["Challenge_Type"].fillna("Unclear")

# change unset values to Affiliation Unknown in challenge init
data["Challenge_Init"] = data["Challenge_Init"].fillna("Affiliation Unknown")

# change unset values to Affiliation Unknown in challenge init
data["Decision"] = data["Decision"].fillna("Unknown")


#drop NaN from months, years, state
data = data.dropna(subset=['Month', "Year", "State"])

In [76]:
# fix months and years
# remove weird labels 
data = data.drop(data[data["Month"] == "22-23"].index)
data = data.drop(data[data["Month"] == "Unclear"].index)
data = data.drop(data[data["Month"] == "Spring"].index)
data = data.drop(data[data["Month"] == "Summer"].index)
data = data.drop(data[data["Month"] == "Fall"].index)

# convert named months to number
months = {month: index for index, month in enumerate(calendar.month_name) if month}
for month, idx in months.items():
    data = data.replace(month, idx)

# make all months and years numeric
data["Month"] = data["Month"].astype("uint8")
data["Year"] = data["Year"].astype("uint16")

In [77]:

# data[data["Decision"] == "Weeded/Deselected"]
data["Decision"] = data["Decision"].replace("Banned From Library", "Banned")
data["Decision"] = data["Decision"].replace("Banned/Removed", "Banned")
data["Decision"] = data["Decision"].replace("Removed", "Banned")

data["Decision"] = data["Decision"].replace("Removed - Curriculum", "Removed From Curriculum")

data["Decision"] = data["Decision"].replace("Retained", "Reshelved")
data["Decision"] = data["Decision"].replace("Returned", "Reshelved")
data["Decision"] = data["Decision"].replace("No Action", "Reshelved")

data["Decision"] = data["Decision"].replace("Challenged - Removed While Under Review", "In Process")
data["Decision"] = data["Decision"].replace("Still in Process", "In Process")
data["Decision"] = data["Decision"].replace("Still in ProcessStill in Process", "In Process")

data["Decision"] = data["Decision"].replace("Retained Restricted", "Restricted")
data["Decision"] = data["Decision"].replace("Retained/Restricted", "Restricted")

data["Decision"] = data["Decision"].replace("Weeded", "Reshelved")
data["Decision"] = data["Decision"].replace("Weeded/Deselected", "Reshelved")

# I dont know what "Lost" Means in this context and only 4 cells have the value
data = data.drop(data[data["Decision"] == '"Lost"'].index)
# data["Decision"] = data["Decision"].replace('"Lost"', "Restricted")

In [78]:
data["Challenge_Init"] = data["Challenge_Init"].replace("School/Library Board Member", "Board Member")
data["Challenge_Init"] = data["Challenge_Init"].replace("FLCA", "Group")

In [79]:
# recompute id field
data["author"] = data["New_Book_Author"]
data = data.drop("New_Book_Author", axis=1)

data["title"] = data["New_Book_Title"]
data = data.drop("New_Book_Title", axis=1)

data["challenge_type"] = data["Challenge_Type"]
data = data.drop("Challenge_Type", axis=1)

data["challenger"] = data["Challenge_Init"]
data = data.drop("Challenge_Init", axis=1)

data["decision"] = data["Decision"]
data = data.drop("Decision", axis=1)

data["library_type"] = data["Library_Type"]
data = data.drop("Library_Type", axis=1)

data["day"] = 1
data["month"] = data["Month"]
data = data.drop("Month", axis=1)
data["year"] = data["Year"]
data = data.drop("Year", axis=1)

data["challenge_date"] = pd.to_datetime(data[["month", "year", "day"]])

data["state"] = data["State"]
data = data.drop("State", axis=1)

In [80]:
data = data.drop("ID", axis=1)
data = data.drop("challenge_type", axis=1)
data = data.drop("challenger", axis=1)
data = data.drop("decision", axis=1)


data.head()


,author,title,library_type,day,month,year,challenge_date,state
0,"Allison, John",Giant Days : Volume 1,School,1,9,2022,2022-09-01,MO
1,"Allison, John",Giant Days: Not On the Test Edition Volume 1 -...,School,1,9,2022,2022-09-01,MO
2,"Allison, John",Giant Days: Not On the Test Edition Vol. 2 - W...,School,1,9,2022,2022-09-01,MO
4,"Barnes, Derrick",The Slumber Party Payback,School,1,9,2021,2021-09-01,PA
6,"Brown, Monica",Lola Levine: Drama Queen,School,1,9,2021,2021-09-01,PA


In [81]:
data.to_pickle("../data/banned_books_clean.pickle")
data.to_csv("../data/banned_books_clean.csv")


In [82]:
data['Count'] = data.apply(lambda row: row.eq("state").sum(), axis=1)

In [88]:
state_counts = data['state'].value_counts()

# Create a new DataFrame from the counts
state_counts_df = pd.DataFrame({'state_abbreviated': state_counts.index, 'count': state_counts.values})



new_row_values = ["DE", 0]
state_counts_df.loc[len(state_counts_df)] = new_row_values
new_row_values = ["KS", 0]
state_counts_df.loc[len(state_counts_df)] = new_row_values



state_counts_df['name'] = state_counts_df['state_abbreviated'].apply(lambda abbr: us.states.lookup(abbr))
state_counts_df = state_counts_df.drop("state_abbreviated", axis=1)
state_counts_df.to_csv("../data/banned_books_states.csv")
display(state_counts_df)

,count,name
0,1713,Florida
1,1572,Texas
2,818,Pennsylvania
3,408,Missouri
4,408,Tennessee
5,235,Kentucky
6,230,Virginia
7,227,South Carolina
8,216,Utah
9,206,North Carolina


In [65]:
import pandas as pd

# Create a sample DataFrame
data = {'Column1': [1, 2, 3],
        'Column2': [10, 20, 30]}
df = pd.DataFrame(data)

print(df)

  Column1  Column2
0       1       10
1       2       20
2       3       30
3       A       40
